In [ ]:

from models.microscopy_cnn import MicroscopyCNN
import torch
from scripts.dataset_loader import MicroscopyDataset
from torch.utils.data import DataLoader

In [ ]:
model = MicroscopyCNN(in_channels=2, task='classification')

# model.load_state_dict(torch.load(r'C:\Users\nmp002\PycharmProjects\MPM-MachineLearning\trained_models\best_NADH_only_24FEB2025.pt', map_location=torch.device('cpu'),weights_only=True))
model.load_state_dict(torch.load(r'C:\Users\nmp002\PycharmProjects\MPM-MachineLearning\trained_models\epoch250_NADH_SHG.pt', map_location=torch.device('cpu'),weights_only=True))

In [ ]:
# Load data
csv_file=r"C:\Users\nmp002\PycharmProjects\MPM-MachineLearning\data\newData\labels.csv"
root_dir=r"C:\Users\nmp002\PycharmProjects\MPM-MachineLearning\data\newData"

dataset = MicroscopyDataset(
    csv_file=csv_file,
    root_dir=root_dir,
    # channels = ['nadh'],
    channels = ['nadh', 'shg'],
    transform=None
)

In [ ]:
specific_test_paths = []
# sample_indices = [23,0,3,20]
sample_indices = range(0,29)
eval_indices = [27, 4, 25, 7, 8]
for sample_index in eval_indices:
    if sample_index not in [25,7]:
        specific_test_paths.append(dataset.sample_wise_paths[sample_index])

print(specific_test_paths)
# print('\n------------------------------------------\n')
# print(dataset.sample_wise_paths)

In [ ]:
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt


def score_em(t, o):
    fpr, tpr, thresholds = roc_curve(t, o)
    score = auc(fpr, tpr)
    score_all = roc_auc_score(t, o)
    print(score_all)
    thresh = thresholds[np.argmax(tpr - fpr)]
    print(type(thresholds), type(thresh))
    preds = [out >= thresh for out in o]
    print(f"Predictions: {preds}")
    for tar,pred in zip(t, preds):
        print(f'{tar} -> {pred}')
    RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=score).plot()
    ConfusionMatrixDisplay.from_predictions(t, preds).plot()

In [ ]:
from sklearn.metrics import auc, roc_curve, confusion_matrix, RocCurveDisplay, ConfusionMatrixDisplay
from scripts.dataset_loader import tiff_to_tensor
import numpy as np
with torch.no_grad():
    model.eval()
    targets = []
    all_targets = []
    outs = []
    all_outs = []
    for sample in specific_test_paths:
        fov_outs = []
        if sample:
            for fov in sample:
                # De-nest fov paths and get the indexed item path
                score = fov[1]
                combined_image = torch.cat([tiff_to_tensor(channel) for channel in fov[0]], dim=0).unsqueeze(0)
                y = model(combined_image).item()
                all_outs.append(y)
                fov_outs.append(y)
                all_targets.append(1 if score > 25 else 0)
            targets.append(1 if score > 25 else 0)
            outs.append(np.min(fov_outs))
# score_em(all_targets, all_outs)
# score_em(targets, outs)

In [ ]:
print(all_targets)
print(outs)

In [ ]:
input_channels = ['nadh','shg']
# Load full dataset
full_dataset = MicroscopyDataset(
    csv_file="../data/newData/labels.csv",
    root_dir="../data/newData",
    channels = input_channels,
    transform=None
)

In [ ]:
samples_list = full_dataset.sample_wise_paths


In [ ]:
import random
random.shuffle(samples_list)
print(samples_list)

In [ ]:
import torchvision.transforms.v2 as tvt
# Transformations for training set
train_transform = tvt.Compose([
    tvt.RandomVerticalFlip(p=0.25),
    tvt.RandomHorizontalFlip(p=0.25),
    tvt.RandomRotation(degrees=(-180, 180))])
full_dataset.transform = train_transform

In [133]:
total_samples = len(samples_list)
train_size = int(0.7 * total_samples)
val_size = int(0.2 * total_samples)
test_size = total_samples - train_size - val_size

indices = torch.utils.data.SubsetRandomSampler(range(len(samples_list)))
print(indices)
indices = [i for i in indices]
print(indices)

[19, 17, 25, 8, 2, 3, 0, 1, 15, 5, 28, 23, 9, 4, 6, 10, 26, 12, 7, 27, 14, 21, 20, 18, 24, 11, 13, 22, 16]


In [ ]:
for i in indices:
    if samples_list[i][0][1] > 25:
        if i+1 < 21:
            print(f"Sample_{(i+1):03} has a HIGH score")
        else:
            print(f"Sample_{(i+2):03} has a HIGH score")
    else:
        if i+1 < 21:
            print(f"Sample_{(i+1):03} has a LOW score")
        else:
            print(f"Sample_{(i+2):03} has a LOW score")

In [ ]:
print(indices)

In [ ]:
sample_id = full_dataset.data_frame['sample_id'].iloc[9]
for i, fov_path in enumerate(full_dataset._denest()):
        print(fov_path[0][0])

In [ ]:
print(indices)
train_samples = indices[:train_size]
print(f"Train samples: {train_samples}")
train_indices = [full_dataset.get_sample_indices(sample) for sample in train_samples]
train_indices = [i for sublist in train_indices for i in sublist]
print(f"Train indices: {train_indices}")
train_data = torch.utils.data.Subset(full_dataset, train_indices)


val_samples = indices[train_size:train_size+val_size]
print(f"Val samples: {val_samples}")
val_indices = [full_dataset.get_sample_indices(sample) for sample in val_samples]
val_indices = [i for sublist in val_indices for i in sublist]
print(f"Val indices: {val_indices}")
val_data = torch.utils.data.Subset(full_dataset, val_indices)

test_samples = indices[train_size+val_size:]
print(f"Test samples: {test_samples}")
test_indices = [full_dataset.get_sample_indices(sample) for sample in test_samples]
test_indices = [i for sublist in test_indices for i in sublist]
print(f"Test indices: {test_indices}")
test_data = torch.utils.data.Subset(full_dataset, test_indices)

In [ ]:
from scripts.dataset_loader import tiff_to_tensor
for i in test_samples:
    for sample_path in full_dataset.sample_wise_paths[i]:
        if sample_path:
            for fov in sample_path:
                score = fov[1]
                combined_image = torch.cat([tiff_to_tensor(channel) for channel in fov[0]], dim=0).unsqueeze(0)

In [ ]:
dataloaders = {
    'train': DataLoader(train_data, batch_size=16, shuffle=True),
    'val': DataLoader(val_data, batch_size=16, shuffle=False),
    'test': DataLoader(test_data, batch_size=len(test_data), shuffle=False)
}

with torch.no_grad():
    model.eval()

    for img, targets in dataloaders['test']:
        print(targets)
        print(img.shape)
        y = model(img).squeeze()

targets = [1 if t > 25 else 0 for t in targets]
print(targets)
print(y)
y = y.numpy().astype(np.float64).tolist()
print(y)
# score_em(targets, y)

In [ ]:
fpr, tpr, thresholds = roc_curve(targets, y)
print(f"fpr: {fpr}, tpr: {tpr}, thresholds: {thresholds}")

In [ ]:
test_score = auc(fpr, tpr)
print(test_score)

In [ ]:
thresh = thresholds[np.argmax(tpr - fpr)]
print(f"Thresh: {thresh}")

In [ ]:
preds = [out >= thresh for out in y]
print(f"Predictions: {preds}")

In [ ]:
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=test_score).plot()

In [ ]:
conf_matrix = ConfusionMatrixDisplay.from_predictions(targets, preds).confusion_matrix

In [ ]:
total_samples = len(samples_list)
train_size = int(0.7 * total_samples)
val_size = int(0.2 * total_samples)
test_size = total_samples - train_size - val_size
print(f"Total samples: {total_samples}, train_size: {train_size}, val_size: {val_size}, test_size: {test_size}")

In [ ]:
train_samples = samples_list[:train_size]
val_samples = samples_list[train_size:train_size+val_size]
test_samples = samples_list[train_size+val_size:]
print(f"Train samples: {train_samples}\n\nval samples: {val_samples}\n\ntest samples: {test_samples}")

In [ ]:
for sample in test_data:
    for fov in sample:
        score = fov[1]
        print(score)